## Making TF records from Solar Data Tutorial

In [1]:
# import necessary packages

#from __future__ import absolute_import, division, print_function
#import argparse
#import multiprocessing
import os
import sys
import numpy as np
#import pdb
from random import shuffle
import pandas as pd
import tensorflow as tf
from astropy.io import fits
from astropy.io.fits import getheader
import tensorflow as tf
from tf_util import example_util
import matplotlib.pylab as plt

In [2]:
# read in the HARPS data that has been shifted and is ready for being added to TF records

# set outfile to filename
outfile = 'HARPS2.3.1_ready_for_TF_records.npz'

# load the file
npzfile = np.load(outfile) 

# List the column names in this file
npzfile.files

['BJD',
 'vrad_star',
 'og_ccf_list',
 'jup_shifted_CCF_data_list',
 'zero_shifted_CCF_list',
 'CCF_normalized_list',
 'cff_residual_list',
 'ccf_residual_rescaled',
 'CCF_normalized_list_cutoff',
 'CCF_residual_list_cutoff',
 'ccf_residual_rescaled_cutoff',
 'mu_og_list',
 'mu_jup_list',
 'mu_zero_list',
 'fwhm',
 'cont',
 'bis',
 'shift_by_rv']

In [ ]:
# We will check which dates did Zoe use in the data set. She excluded multiple dates, Why?

In [8]:
npzfile['CCF_normalized_list']

array([[1.0025297 , 1.00104516, 0.99953513, ..., 1.00245604, 1.00279631,
        1.00303837],
       [1.00239219, 1.0008953 , 0.99936542, ..., 1.00261197, 1.00303866,
        1.0032854 ],
       [1.00241965, 1.00098246, 0.99946956, ..., 1.00256702, 1.00295503,
        1.00318626],
       ...,
       [1.00226386, 1.00083192, 0.99934611, ..., 1.0025985 , 1.00294911,
        1.00314657],
       [1.00230273, 1.00080272, 0.99935437, ..., 1.00247513, 1.0028813 ,
        1.00307449],
       [1.00228099, 1.00073469, 0.99930664, ..., 1.00253727, 1.00293112,
        1.00314595]], shape=(528, 49))

In [3]:
from datetime import datetime, timedelta
def BJD2dates(date_values):
    """
    Accepts an array of float dates (JD or MJD), sorts them, 
    and prints the readable ISO format.
    """
    # Ensure input is a numpy array and sort it
    dates_arr = np.sort(np.array(date_values, dtype=np.float64))
    
    # MJD Epoch (November 17, 1858)
    epoch = datetime(1858, 11, 17)
    
    # Header for the output
    print(f"{'Original Value':<18} | {'Type':<5} | {'Readable Date (UTC/ISO)'}")
    print("-" * 60)
    
    for val in dates_arr:
        # Determine if the value is JD or MJD
        if val > 2400000:
            # Convert JD to MJD
            mjd_value = val - 2400000.5
            date_type = "JD"
        else:
            # Already MJD
            mjd_value = val
            date_type = "MJD"

        # Convert to datetime object
        # timedelta handles the fractional days (time) automatically
        readable_date = epoch + timedelta(days=float(mjd_value))
        
        # Print formatted row
        print(f"{val:<18.6f} | {date_type:<5} | {readable_date.isoformat()}")

In [4]:
BJD2dates(npzfile['BJD'])

Original Value     | Type  | Readable Date (UTC/ISO)
------------------------------------------------------------
2457222.995876     | JD    | 2015-07-19T11:54:03.645345
2457232.992152     | JD    | 2015-07-29T11:48:41.902531
2457233.996867     | JD    | 2015-07-30T11:55:29.298672
2457235.000008     | JD    | 2015-07-31T12:00:00.666904
2457236.000665     | JD    | 2015-08-01T12:00:57.423547
2457237.000425     | JD    | 2015-08-02T12:00:36.751480
2457237.996225     | JD    | 2015-08-03T11:54:33.819754
2457239.032385     | JD    | 2015-08-04T12:46:38.050806
2457239.977159     | JD    | 2015-08-05T11:27:06.579925
2457242.032933     | JD    | 2015-08-07T12:47:25.433973
2457252.004563     | JD    | 2015-08-17T12:06:34.219266
2457253.991040     | JD    | 2015-08-19T11:47:05.875251
2457256.012939     | JD    | 2015-08-21T12:18:37.917010
2457257.029882     | JD    | 2015-08-22T12:43:01.763506
2457260.016671     | JD    | 2015-08-25T12:24:00.364587
2457261.013146     | JD    | 2015-08-26T12:18:

In [13]:
# Define the 
class TfRecordMaker:

    def __init__(self, input_path, path, numfits, index=None):
        self.input_path = input_path or os.input_path.dirname(os.input_path.realpath(__file__))#or 'shifted_fits_clean73_May26_one_file/'  # or os.path.dirname(os.path.realpath(__file__))
        self.path = path or os.path.dirname(os.path.realpath(__file__))
        self.numfits = numfits or 0
        self.index = index

    def make_examples(self):
        examples = []
        print(self.index)
        
        index_number = 0
        # index = np.arange(0,self.numfits,1)
        # np.random.seed(42)
        # np.random.shuffle(index)
        
        
        # read in files
        npzfile = np.load(outfile) 
        

        for j in self.index:
            ex = tf.train.Example()

            # Set CCF features.
            example_util.set_float_feature(ex, "OG_CCF",
                                           npzfile['og_ccf_list'][j])
            example_util.set_float_feature(ex, "JUP_CCF",
                                           npzfile['jup_shifted_CCF_data_list'][j])
            example_util.set_float_feature(ex, "ZERO_CCF",
                                           npzfile['zero_shifted_CCF_list'][j])
            example_util.set_float_feature(ex, "CCF",
                                           npzfile['CCF_normalized_list'][j]) 
            example_util.set_float_feature(ex, "CCF_residuals",
                                           npzfile['cff_residual_list'][j]) 
            example_util.set_float_feature(ex, "Rescaled CCF_residuals",
                                           npzfile['ccf_residual_rescaled'][j]) 
            example_util.set_float_feature(ex, "CCF_cutoff",
                                           npzfile['CCF_normalized_list_cutoff'][j]) 
            example_util.set_float_feature(ex, "CCF_residuals_cutoff",
                                           npzfile['CCF_residual_list_cutoff'][j]) 
            example_util.set_float_feature(ex, "Rescaled CCF_residuals_cutoff",
                                           npzfile['ccf_residual_rescaled_cutoff'][j]) 

            # prints what iteration we are currently at
            index_number = index_number + 1
            if index_number % 500 == 0:
                print(index_number)

            # Set residuals
            median_rv = np.median(npzfile['vrad_star'])
            # example_util.set_feature(ex, "activity signal residuals", act_signal)
            example_util.set_feature(ex, "activity signal", [(npzfile['vrad_star'][j] - median_rv)])  # in km/s
            example_util.set_feature(ex, "mu_og_fit", [(npzfile['mu_og_list'][j])])
            example_util.set_feature(ex, "mu_jup_fit", [(npzfile['mu_jup_list'][j])])
            example_util.set_feature(ex, "mu_zero_fit", [(npzfile['mu_zero_list'][j])])
            example_util.set_feature(ex, "BJD", [(npzfile['BJD'][j])])
            example_util.set_feature(ex, "fwhm", [(npzfile['fwhm'][j])])
            example_util.set_feature(ex, "contrast", [(npzfile['cont'][j])])
            example_util.set_feature(ex, "bis", [(npzfile['bis'][j])])

            # set the other features in the header
            #for k in headr_all[j]:
            #    example_util.set_feature(ex, str(k), [headr_all[j][k]])

            examples.append(ex)
        return examples

def tf_writer(input_path, path, numfits, randseed):
    num_ccfs = 528
    full_val_cutoff = int(0.80*num_ccfs) # where 628 is the number of nonzero ccfs
    cross_val_cutoff = int(0.08 * num_ccfs)
    val_cutoff = int(0.1*num_ccfs)
    test_cutoff = int(0.1*num_ccfs)
    index = np.arange(0, num_ccfs, 1)
    np.random.seed(randseed)
    np.random.shuffle(index)

    reps_bf = []
    reps_aft = []
    train_indeces = []
    intervals = [0.08, 0.16, 0.24, 0.32, 0.40, 0.48, 0.56, 0.64, 0.72, 0.80] # fix this (0.08 each) so it's 0.08*10 = 0.80
    for i in range(0, len(intervals)):
        if intervals[i] != 0.08:
            reps_bf.append(int(intervals[i - 1] * num_ccfs))
            reps_aft.append(int(intervals[i] * num_ccfs))
            train_indeces.append(index[int(intervals[i - 1] * num_ccfs):int(intervals[i] * num_ccfs)])
        else:
            print(intervals[i])
            reps_bf.append(0)
            reps_aft.append(int(intervals[i] * num_ccfs))
            train_indeces.append(index[0:int(intervals[i] * num_ccfs)])

    subset0 = train_indeces[1:]
    subset1 = train_indeces[0:1] + train_indeces[2:]
    subset2 = train_indeces[0:2] + train_indeces[3:]
    subset3 = train_indeces[0:3] + train_indeces[4:]
    subset4 = train_indeces[0:4] + train_indeces[5:]
    subset5 = train_indeces[0:5] + train_indeces[6:]
    subset6 = train_indeces[0:6] + train_indeces[7:]
    subset7 = train_indeces[0:7] + train_indeces[8:]
    subset8 = train_indeces[0:8] + train_indeces[9:]
    subset9 = train_indeces[0:9]

    flattened0 = [val for sublist in subset0 for val in sublist]
    flattened1 = [val for sublist in subset1 for val in sublist]
    flattened2 = [val for sublist in subset2 for val in sublist]
    flattened3 = [val for sublist in subset3 for val in sublist]
    flattened4 = [val for sublist in subset4 for val in sublist]
    flattened5 = [val for sublist in subset5 for val in sublist]
    flattened6 = [val for sublist in subset6 for val in sublist]
    flattened7 = [val for sublist in subset7 for val in sublist]
    flattened8 = [val for sublist in subset8 for val in sublist]
    flattened9 = [val for sublist in subset9 for val in sublist]
    indexes_full_val = [val for sublist in train_indeces for val in sublist]

    full_train_flats = []
    full_train_flats.extend([flattened0, flattened1, flattened2, flattened3, flattened4, flattened5, flattened6, flattened7, flattened8, flattened9])

    #train_index = index[0:train_cutoff]
    val_index = index[int(0.8 * num_ccfs):int(0.9 * num_ccfs)]
    test_index = index[int(0.9 * num_ccfs):]

    # # loop through cross_val sets
    # for iteration in range(0, len(full_train_flats)):
    #     with tf.python_io.TFRecordWriter('Archive_HARPS_N/TF_record_Jul_8/TF_ccf_train'+str(iteration)) as writer:
    #         tf_record_maker = TfRecordMaker(path=path, numfits=numfits, index=np.array(full_train_flats[iteration]))
    #         number_examples_train = 0
    #         examples_tf = tf_record_maker.make_examples()
    #         for example in examples_tf[0:train_cutoff]:
    #             print("train")
    #             number_examples_train = number_examples_train + 1
    #             if number_examples_train % 100 == 0:
    #                 print("iteration for training set: " + str(number_examples_train))
    #             writer.write(example.SerializeToString())
    #             # print(ex)

    # Make directory if it does not exist
    if not os.path.exists(path):
        os.makedirs(path)

    for iteration in range(0, len(train_indeces)):
        with tf.io.TFRecordWriter(path+'TF_ccf_cross_val'+str(iteration)) as writer:
             tf_record_maker = TfRecordMaker(input_path=input_path, path=path, numfits=numfits, index=train_indeces[iteration])
             number_examples_val = 0
             eval_counter = 0
             #for example in tf_record_maker.make_examples()[train_cutoff:val_cutoff]:
             for example in tf_record_maker.make_examples()[0:cross_val_cutoff+1]:
                 eval_counter += 1
                 print("val: " + str(eval_counter))

                 number_examples_val = number_examples_val + 1
                 if number_examples_val%100 == 0:
                     print("iteration for evaluation set: "+str(number_examples_val))
                 writer.write(example.SerializeToString())

    with tf.io.TFRecordWriter(path+'TF_ccf_val') as writer:
        tf_record_maker = TfRecordMaker(input_path=input_path, path=path, numfits=numfits, index=val_index)
        number_examples_test = 0
        test_counter = 0
        for example in tf_record_maker.make_examples()[0:val_cutoff+1]:
        #for example in tf_record_maker.make_examples()[val_cutoff:]:
            test_counter += 1
            print("test: " + str(test_counter))
            number_examples_test = number_examples_test + 1
            if number_examples_test % 100 == 0:
                print("iteration for testing set: "+str(number_examples_test))
            writer.write(example.SerializeToString())
            # print(ex)


    with tf.io.TFRecordWriter(path+'TF_ccf_test') as writer:
        tf_record_maker = TfRecordMaker(input_path=input_path, path=path, numfits=numfits, index=test_index)
        number_examples_test = 0
        test_counter = 0
        for example in tf_record_maker.make_examples()[0:test_cutoff+1]:
        #for example in tf_record_maker.make_examples()[val_cutoff:]:
            test_counter += 1
            print("test: " + str(test_counter))
            number_examples_test = number_examples_test + 1
            if number_examples_test % 100 == 0:
                print("iteration for testing set: "+str(number_examples_test))
            writer.write(example.SerializeToString())
            # print(ex)

    # Optional: also write a file with all the evaluation files in one file
    #full_val_cutoff
    #indexes_full_val

    with tf.io.TFRecordWriter(path+'TF_ccf_full_train') as writer:
        tf_record_maker = TfRecordMaker(input_path=input_path, path=path, numfits=numfits, index=indexes_full_val)
        number_examples_val = 0
        eval_counter = 0
        # for example in tf_record_maker.make_examples()[train_cutoff:val_cutoff]:
        for example in tf_record_maker.make_examples()[0:full_val_cutoff+1]:
            eval_counter += 1
            print("val: " + str(eval_counter))
            number_examples_val = number_examples_val + 1
            if number_examples_val % 100 == 0:
                print("iteration for evaluation set: " + str(number_examples_val))
            writer.write(example.SerializeToString())
            # print(ex)

In [14]:
tf_writer(input_path='/Users/zdebeurs/Documents/Github/rv_net',
          path="TF_records_June2023/",
          randseed=20,
          numfits=528)

0.08
[ 34 197 333 314 284 221  23 510 121 437 356 206 211 169 498  31 419 318
 362 260  63 272 457 217 363 336 523  30 140  81  94 263  56  10 129  80
 351  35  29 470  91 513]
val: 1
val: 2
val: 3
val: 4
val: 5
val: 6
val: 7
val: 8
val: 9
val: 10
val: 11
val: 12
val: 13
val: 14
val: 15
val: 16
val: 17
val: 18
val: 19
val: 20
val: 21
val: 22
val: 23
val: 24
val: 25
val: 26
val: 27
val: 28
val: 29
val: 30
val: 31
val: 32
val: 33
val: 34
val: 35
val: 36
val: 37
val: 38
val: 39
val: 40
val: 41
val: 42
[293  90 405  71 422 525 379 179  13 189 409 229 511 421 432 466 165  17
 237 243 410 343 156 145 228 403  14 397 467 147 257 454 166 172 254   7
 204 468 423   9   2 364]
val: 1
val: 2
val: 3
val: 4
val: 5
val: 6
val: 7
val: 8
val: 9
val: 10
val: 11
val: 12
val: 13
val: 14
val: 15
val: 16
val: 17
val: 18
val: 19
val: 20
val: 21
val: 22
val: 23
val: 24
val: 25
val: 26
val: 27
val: 28
val: 29
val: 30
val: 31
val: 32
val: 33
val: 34
val: 35
val: 36
val: 37
val: 38
val: 39
val: 40
val: 41
val: 

## Try adding a new feature now :)